In [2]:
import os
import pandas as pd
import yaml
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"

from IPython.display import Markdown, display
os.chdir("..")
import visualizer_helpers as vh

In [3]:
# NOTE: SPECIALLY TAGGED PARAMETERS CELL
# Assignments in this cell may be overwritten at compile-time
# Otherwise, the below defaults to not filtering any zones
zone_set: str = 'all'
how_method: str = 'any'
affected_tazs: list = []
affected_mazs: list = []

In [4]:
# get corresponding filter from zone_set parameter
single_filter_tazs, multi_filter_tazs = vh.get_filters(zone_set, how_method, affected_tazs)
single_filter_mazs, multi_filter_mazs = vh.get_filters(zone_set, how_method, affected_mazs)

In [10]:
# file IO locations
with open('_quarto.yml') as f:
    config = yaml.safe_load(f)
base_dir = config['sources']['base']
build_dir = config['sources']['build']

In [11]:
base_tour = pd.read_csv(f"{base_dir}/final_tours.csv", 
                        index_col='tour_id', 
                        usecols=['tour_id','tour_category','origin','destination','primary_purpose', 'tour_mode'])

base_tour_idx = base_tour.index

build_tour = pd.read_csv(f"{build_dir}/final_tours.csv", 
                         index_col='tour_id', 
                         usecols=['tour_id','tour_category','origin','destination','primary_purpose', 'tour_mode'])


In [12]:
print(f"""Filtering tours by origin {
    'and' if how_method == 'all' else 'or' if how_method == 'any' else '???'
    } destination TAZ.\nOriginal tours in base: {len(base_tour)}\tbuild: {len(build_tour)}""")

base_tour = base_tour[multi_filter_mazs([base_tour.origin, base_tour.destination])] # base tour in the filtered set

# special build-case filtering
build_tour = build_tour[(
    build_tour.index.isin(base_tour_idx) # originally existed in base
    & build_tour.index.isin(base_tour.index)    # and was in the current set for the base
    ) |     # OR
    ((~build_tour.index.isin(base_tour_idx)) # is a new tour 
     & multi_filter_mazs([build_tour.origin, build_tour.destination]) # and it's in this set
     )]

print(f"After filtering, tours in base: {len(base_tour)}\tbuild: {len(build_tour)}")

Filtering tours by origin or destination TAZ.
Original tours in base: 49	build: 49
After filtering, tours in base: 49	build: 49


In [13]:
name_dict = {'escort': "Escorting", 
             'work': "Work", 
             'school': "School", 
             'othmaint': "Other-Maintenance",
             'othdiscr': "Other-Discretionary", 
             'shopping': "Shopping",
             'atwork': "At-work Subtour", 
             'eatout': "Eat Out", 
             'social' : "Social", 
             'univ': "University"}

## Non-Mandatory Tour Frequency

In [ ]:
print("Lorem ipsum") # Placeholder for WSP

## Non-Mandatory Tour Destinations

In [28]:
df = base_tour[base_tour.tour_category == 'non_mandatory'].merge(build_tour[build_tour.tour_category == 'non_mandatory'],
                                                                 how='outer',
                                                                 left_index=True,
                                                                 right_index=True,
                                                                 suffixes=('_base','_build'))



In [29]:
df.loc[df.destination_base == df.destination_build,'Case'] = "Unchanged"
df.loc[df.destination_base != df.destination_build,'Case'] = "Changed"

df.loc[df.destination_base.isna(),'Case'] = "Newly Created"
df.loc[df.destination_build.isna(),"Case"] = "Removed"


In [30]:
df = df.Case.value_counts()

df

Case
Unchanged    30
Name: count, dtype: int64

In [31]:
fig = vh.create_pie_chart(df.to_frame().sort_index(),["count"])
fig.show()

In [32]:
# | output: asis
# above comment is needed for Quarto to render subtabs correctly

for key, value in name_dict.items():

    if len(base_tour[
        (base_tour.tour_category == 'non_mandatory')
        & (base_tour.primary_purpose == key)
        ]) == 0:
        continue
    
    print(f"#### {value}")
    df = base_tour[
        (base_tour.tour_category == 'non_mandatory')
        & (base_tour.primary_purpose == key)
        ].merge(build_tour[
                    (build_tour.tour_category == 'non_mandatory')
                    & (build_tour.primary_purpose == key)
                ],
                how='left',
                left_index=True,
                right_index=True,
                suffixes=('_base','_build'))
    
    df.loc[df.destination_base == df.destination_build,'Case'] = "Unchanged"
    df.loc[df.destination_base != df.destination_build,'Case'] = "Changed"

    df.loc[df.destination_base.isna(),'Case'] = "Newly Created"
    df.loc[df.destination_build.isna(),"Case"] = "Removed"

    
    df = df.Case.value_counts()

    print(str(df).replace("\n","<br>"))
    
    fig = vh.create_pie_chart(df.to_frame().sort_index(),["count"])
    
    fig.show()
    display(Markdown(" "))

#### Escorting
Case<br>Unchanged    8<br>Name: count, dtype: int64


#### Other-Maintenance
Case<br>Unchanged    4<br>Name: count, dtype: int64


#### Other-Discretionary
Case<br>Unchanged    9<br>Name: count, dtype: int64


#### Shopping
Case<br>Unchanged    5<br>Name: count, dtype: int64


#### Eat Out
Case<br>Unchanged    3<br>Name: count, dtype: int64


#### Social
Case<br>Unchanged    1<br>Name: count, dtype: int64


## Non-Mandatory Tour Scheduling

In [ ]:
print("Lorem ipsum") # Placeholder for WSP

## Tour Mode Choice

In [15]:
# xtab base-vs build mode choice for all NM tours
# xtab base-vs build mode choice for NM tours by purpose

In [52]:
df = pd.merge(
    base_tour[base_tour.tour_category == "non_mandatory"],
    build_tour[build_tour.tour_category == "non_mandatory"],
    how='outer',
    left_index=True,
    right_index=True,
    suffixes=['_base', '_build']
)

df.loc[df.tour_mode_base == df.tour_mode_build,'Case'] = "Unchanged"
df.loc[df.tour_mode_base != df.tour_mode_build,'Case'] = "Changed"

df.loc[df.tour_mode_base.isna(),'Case'] = "Newly Created"
df.loc[df.tour_mode_build.isna(),"Case"] = "Removed"

df.Case.value_counts()

Case
Unchanged    30
Name: count, dtype: int64

In [53]:
fig = vh.create_pie_chart(df.Case.value_counts().to_frame().sort_index(), ["count"])
fig

In [54]:
xtab = df[["tour_mode_base", "tour_mode_build"]].value_counts(dropna=False).unstack().fillna(0)
xtab

tour_mode_build,BIKE,DRIVEALONE,SHARED2,SHARED3,WALK,WALK_LOC
tour_mode_base,,,,,,
BIKE,2.0,0.0,0.0,0.0,0.0,0.0
DRIVEALONE,0.0,11.0,0.0,0.0,0.0,0.0
SHARED2,0.0,0.0,1.0,0.0,0.0,0.0
SHARED3,0.0,0.0,0.0,3.0,0.0,0.0
WALK,0.0,0.0,0.0,0.0,12.0,0.0
WALK_LOC,0.0,0.0,0.0,0.0,0.0,1.0


In [74]:
for key, value in name_dict.items():
    df_purp = df.loc[df.primary_purpose_base == key]
    if len(df_purp) == 0:
        continue 
        
    print(f"#### {value}")
    df_purp_cases = df_purp.Case.value_counts()
    print(str(df_purp_cases).replace("\n","<br>"))
    
    fig = vh.create_pie_chart(df_purp_cases.to_frame().sort_index(),["count"])
    
    fig.show()
    display(Markdown(" "))

#### Escorting
Case<br>Unchanged    8<br>Name: count, dtype: int64


#### Other-Maintenance
Case<br>Unchanged    4<br>Name: count, dtype: int64


#### Other-Discretionary
Case<br>Unchanged    9<br>Name: count, dtype: int64


#### Shopping
Case<br>Unchanged    5<br>Name: count, dtype: int64


#### Eat Out
Case<br>Unchanged    3<br>Name: count, dtype: int64


#### Social
Case<br>Unchanged    1<br>Name: count, dtype: int64


## At-Work Subtour Frequency

In [ ]:
print("Lorem ipsum") # Placeholder for WSP

## At-Work Subtour Destinations

In [ ]:
base_tour = pd.read_csv(f"{base_dir}/final_tours.csv", 
                        index_col='tour_id', 
                        usecols=['tour_id','tour_category','origin','destination','primary_purpose'])

build_tour = pd.read_csv(f"{build_dir}/final_tours.csv", 
                         index_col='tour_id', 
                         usecols=['tour_id','tour_category','origin','destination','primary_purpose'])


In [ ]:
print(f"""Filtering tours by origin {
    'and' if how_method == 'all' else 'or' if how_method == 'any' else '???'
    } destination TAZ.\nOriginal tours in base: {len(base_tour)}\tbuild: {len(build_tour)}""")

base_tour = base_tour[multi_filter_mazs([base_tour.origin, base_tour.destination])] # base tour in the filtered set

# special build-case filtering
build_tour = build_tour[(
    build_tour.index.isin(base_tour_idx) # originally existed in base
    & build_tour.index.isin(base_tour.index)    # and was in the current set for the base
    ) |     # OR
    ((~build_tour.index.isin(base_tour_idx)) # is a new tour 
     & multi_filter_mazs([build_tour.origin, build_tour.destination]) # and it's in this set
     )]

print(f"After filtering, tours in base: {len(base_tour)}\tbuild: {len(build_tour)}")

In [ ]:
df = base_tour[base_tour.tour_category == 'atwork'].merge(build_tour[build_tour.tour_category == 'atwork'],
                                                          how='outer',
                                                          left_index=True,
                                                          right_index=True,
                                                          suffixes=('_base','_build'))


In [ ]:
df.loc[df.destination_base == df.destination_build,'Case'] = "Unchanged"
df.loc[df.destination_base != df.destination_build,'Case'] = "Changed"

df.loc[df.destination_base.isna(),'Case'] = "Newly Created"
df.loc[df.destination_build.isna(),"Case"] = "Removed"


In [ ]:
df = df.Case.value_counts()

df

In [ ]:
fig = vh.create_pie_chart(df.to_frame().sort_index(),["count"])
fig.show()

## At-Work Subtour Scheduling

In [ ]:
print("Lorem ipsum") # Placeholder for WSP

## At-Work Subtour Mode Choice

In [ ]:
print("Lorem ipsum") # Placeholder for CS

In [1]:
dfx = pd.read_csv(f"{base_dir}/final_tours.csv", 
                        index_col='tour_id')


NameError: name 'pd' is not defined

## Stop Frequency

In [ ]:
print("Lorem ipsum") # Placeholder for CS